In [36]:
import os
import pandas as pd
import numpy as np
import json
import seaborn as sns
import funcTest as ft
import funcData as fd
from datetime import datetime

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [37]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
path_rawdata    = 'F://TFG//datasets/raw_datasets//'
path_train      = 'F://TFG//datasets//data_train//'
path_footdata   ='F:\\TFG\\datasets\\football-data\\'

In [39]:
rawdata         = pd.read_csv(path_rawdata+'rawDB.csv')
footdata        = pd.read_csv(path_rawdata+'fdataDB.csv')
partidos_raw    = pd.read_json(path_rawdata+'RAW_partidos.json')
fdata_allseasons   = pd.read_csv(path_rawdata+'fdata_allseasons.csv',sep=';')
data = pd.read_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';',index_col='wyId')

with open('F:\\TFG\\datasets\\data_train\\equipos.json','r') as equipos_json:
    equipos = json.load(equipos_json)

In [40]:
from random import randint
equipos['name2id_wysc'].get(equipos['foot2wysc_name'].get('Paris SG',-1), randint(0,1000))

3767

##### Identificar equipos (id) y unir datasets

In [41]:
# encontrar los mismos partidos en ambos datasets
    # formatear las fechas de los partidos y id de los equipos para identificarlos
# añadir id de los equipos (y partidos si hay) en fdata_allseasons
cont = 4000

def getId(name):
    global cont, equipos
    id = equipos['name2id_wysc'].get(equipos['foot2wysc_name'].get(name,-1), -1)
    if id==-1:
        equipos['foot2wysc_name'][name] = name
        equipos['id2name_wysc'][cont] = name
        equipos['name2id_wysc'][name] = cont
        id = cont; cont += 1
    return id

In [42]:
fdata_allseasons['FTR'][fdata_allseasons.FTR=='H'] = 1
fdata_allseasons['FTR'][fdata_allseasons.FTR=='D'] = 0
fdata_allseasons['FTR'][fdata_allseasons.FTR=='A'] = 2

fdata_allseasons['IdH'] = [getId(name) for name in fdata_allseasons.HomeTeam]
fdata_allseasons['IdA'] = [getId(name) for name in fdata_allseasons.AwayTeam]

##### Identificar partidos con ID Wyscout

In [43]:
fdata1718 = fdata_allseasons[fdata_allseasons.season==1718].reset_index().set_index(['IdH','IdA'])
dict_teams2wyId = data.reset_index().set_index(['teamId_home','teamId_away']).wyId.to_dict()
index_transformed = np.maximum(
            np.nan_to_num(fdata1718.index.map(dict_teams2wyId).to_numpy(), nan=-1),fdata1718['index'].to_numpy())

fdata_allseasons['aux'] = fdata_allseasons.index
fdata_allseasons.reset_index(drop=True)
fdata_allseasons['aux'][fdata_allseasons.season==1718] = index_transformed.astype(int)
fdata_allseasons.index = fdata_allseasons.aux
fdata_allseasons.drop(columns='aux',inplace=True)

##### Formatear fechas

In [44]:
def formatDatesDF(df):
    def formatDate(strdate):
        try: date = datetime.strptime(strdate, "%d/%m/%y").date()
        except: date = datetime.strptime(strdate, "%d/%m/%Y").date()
        return date
    df['Date'] = [formatDate(d) for d in df.Date]

##### Limpieza del dataset

In [45]:
len(fdata_allseasons[fdata_allseasons.isna().any(axis=1)]) / len(fdata_allseasons)

# aqui vemos que la mayoria de filas tienen algun componente NaN

0.9725192988099067

In [46]:
print(len(fdata_allseasons[fdata_allseasons.HO.isna()]) / len(fdata_allseasons))

# vemos que los atributos de HO y AO (offside) tienen casi todos sus filas nulas

fdata_allseasons_clean = fdata_allseasons.drop(['HO','AO'],axis=1)
print(len(fdata_allseasons_clean[fdata_allseasons_clean.isna().any(axis=1)]) / len(fdata_allseasons_clean))

0.9724187841749758
0.9725192988099067


In [47]:
# LIMPIEZA:
# Atributos que tienen casi todas sus filas nulas o gran parte de ellas
# Filas que contienen muchos atributos con valor nulo
def clean_df(df):
    for col in df.columns:
        ratio_nan = len(df[df[col].isna()]) / len(df)
        print(col,ratio_nan)
        if ratio_nan > 0.6: df.drop(col,axis=1,inplace=True)

In [48]:
clean_df(fdata_allseasons)

Div 0.08766886458668383
Date 0.08766886458668383
HomeTeam 0.08766886458668383
AwayTeam 0.08766886458668383
FTHG 0.08766886458668383
FTAG 0.08766886458668383
FTR 0.08766886458668383
HTHG 0.14691219041492443
HTAG 0.14691219041492443
HTR 0.14691219041492443
Attendance 0.9724991958829206
Referee 0.8278988420714056
HS 0.37508041170794465
AS 0.37508041170794465
HST 0.38738340302348023
AST 0.38738340302348023
HHW 0.9724187841749758
AHW 0.9724187841749758
HC 0.3826190093277581
AC 0.3826190093277581
HF 0.39029832743647475
AF 0.39029832743647475
HO 0.9724187841749758
AO 0.9724187841749758
HY 0.3747989707301383
AY 0.3747788678031521
HR 0.3747788678031521
AR 0.3747788678031521
season 0.0
IdH 0.0
IdA 0.0


In [49]:
fdata_allseasons_clean = fdata_allseasons.loc[~fdata_allseasons.FTHG.isna()]
fdata_allseasons_clean['matchId'] = fdata_allseasons_clean.index
clean_df(fdata_allseasons_clean)

Div 0.0
Date 0.0
HomeTeam 0.0
AwayTeam 0.0
FTHG 0.0
FTAG 0.0
FTR 0.0
HTHG 0.0649362095939008
HTAG 0.0649362095939008
HTR 0.0649362095939008
HS 0.31502985699491
AS 0.31502985699491
HST 0.3285150827402331
AST 0.3285150827402331
HC 0.323292862966309
AC 0.323292862966309
HF 0.33171011171584075
AF 0.33171011171584075
HY 0.3147213714386444
AY 0.31469933675605405
HR 0.31469933675605405
AR 0.31469933675605405
season 0.0
IdH 0.0
IdA 0.0
matchId 0.0


##### Ultimos N partidos

In [50]:
def ult_partidos(df,npj=4):
    df.sort_values('Date',axis=0,ascending=False,inplace=True)
    last_n_matches = []
    df = df[['Div','season','Date','IdH','IdA','matchId']]
    dict_df_seasons = {k:df.loc[df.season==k] for k in np.unique(df.season)}

    def getMatchesTeam(df,team,n,home=True,away=True):
        # home y away indican si tomamos en cuenta los partidos locales y visitantes
        mask = (home & np.array(df.IdH==team)) | (away & np.array(df.IdA==team))
        if np.sum(mask) < n: return [np.nan, np.nan]
        return np.array(df[mask].sort_values('Date',axis=0,ascending=False).matchId[:n])

    for match in df.itertuples():
        div = match.Div; season = match.season; date = match.Date
        selDF = dict_df_seasons[season]
        selDF = selDF.loc[selDF.Div==div]
        selDF = selDF.loc[selDF.Date < date]
        last_n_matches.append([getMatchesTeam(selDF,match.IdH,npj), 
                                getMatchesTeam(selDF,match.IdA,npj)])

    return np.array(last_n_matches)

In [22]:
# ult_partidos(historical_goals,npj=4)

#### Historical dataset: only goals

In [295]:
fdata_allseasons_goals = fdata_allseasons_clean.dropna(axis=1)

In [296]:
for div in np.unique(fdata_allseasons_goals.Div):
    print(div, len(fdata_allseasons_goals[fdata_allseasons_goals.Div==div]))

print()

for season in np.unique(fdata_allseasons_goals.season):
    print(season, len(fdata_allseasons_goals[fdata_allseasons_goals.season==season]))

D1 7938
E0 10014
F1 7889
I1 9118
SP1 10424

1 1678
102 1372
203 1372
304 380
405 1066
506 1446
607 1826
708 1826
809 1826
910 1826
1011 1826
1112 1826
1213 1826
1314 1826
1415 1826
1516 1826
1617 1826
1718 1826
1819 1826
1920 1725
2021 1756
9394 1454
9495 1454
9596 1454
9697 1454
9798 1678
9899 1678
9900 1678


In [297]:
formatDatesDF(fdata_allseasons_goals)
fdata_allseasons_goals

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,matchId
aux,,,,,,,,,,,
0,E0,2000-08-19,Charlton,Man City,4.0,0.0,1,1,4000,1625,0
1,E0,2000-08-19,Chelsea,West Ham,4.0,2.0,1,1,1610,1633,1
2,E0,2000-08-19,Coventry,Middlesbrough,1.0,3.0,2,1,4001,4007,2
3,E0,2000-08-19,Derby,Southampton,2.0,2.0,0,1,4002,1619,3
4,E0,2000-08-19,Leeds,Everton,2.0,0.0,1,1,4003,1623,4
...,...,...,...,...,...,...,...,...,...,...,...
49493,F1,2000-05-13,Nancy,Auxerre,2.0,0.0,1,9900,4123,4115,49493
49494,F1,2000-05-13,Rennes,Metz,2.0,0.0,1,9900,3774,3777,49494
49495,F1,2000-05-13,Sedan,Marseille,2.0,2.0,0,9900,4116,3771,49495


In [298]:
# fdata_allseasons_goals['IdH'] = [getId(name) for name in fdata_allseasons_goals.HomeTeam]
# fdata_allseasons_goals['IdA'] = [getId(name) for name in fdata_allseasons_goals.AwayTeam]

In [299]:
# sustituir Gladbach y Villareal
# 4078 por 2454
# 4057 por 682
fdata_allseasons_goals.IdH[fdata_allseasons_goals.IdH==4078] = 2454
fdata_allseasons_goals.IdA[fdata_allseasons_goals.IdA==4078] = 2454
fdata_allseasons_goals.IdH[fdata_allseasons_goals.IdH==4057] = 682
fdata_allseasons_goals.IdA[fdata_allseasons_goals.IdA==4057] = 682

In [300]:
def getPoints(scored,received):
    if scored>received: return 3
    if scored==received: return 1
    else: return 0

In [301]:
fdata_allseasons_goals

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,matchId
aux,,,,,,,,,,,
0,E0,2000-08-19,Charlton,Man City,4.0,0.0,1,1,4000,1625,0
1,E0,2000-08-19,Chelsea,West Ham,4.0,2.0,1,1,1610,1633,1
2,E0,2000-08-19,Coventry,Middlesbrough,1.0,3.0,2,1,4001,4007,2
3,E0,2000-08-19,Derby,Southampton,2.0,2.0,0,1,4002,1619,3
4,E0,2000-08-19,Leeds,Everton,2.0,0.0,1,1,4003,1623,4
...,...,...,...,...,...,...,...,...,...,...,...
49493,F1,2000-05-13,Nancy,Auxerre,2.0,0.0,1,9900,4123,4115,49493
49494,F1,2000-05-13,Rennes,Metz,2.0,0.0,1,9900,3774,3777,49494
49495,F1,2000-05-13,Sedan,Marseille,2.0,2.0,0,9900,4116,3771,49495


In [308]:
kk1 = fdata_allseasons_goals[['IdH','IdA']].drop_duplicates().reset_index(drop=True)
kk2 = kk1.copy()
kk2.columns = ['IdA','IdH']
kk = pd.DataFrame({'IdH':[*kk1.IdH,*kk2.IdH], 'IdA':[*kk1.IdA,*kk2.IdA], 'id_derby':[*kk1.index,*kk2.index]}).drop_duplicates()

In [309]:
kkg = kk.groupby('id_derby').count()
(kkg.IdH!=2).sum()

0

In [310]:
fdata_allseasons_goals = fdata_allseasons_goals.merge(kk,on=['IdH','IdA'],how='left')[::2]

In [311]:
fdata_allseasons_goals

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,matchId,id_derby
0,E0,2000-08-19,Charlton,Man City,4.0,0.0,1,1,4000,1625,0,0
2,E0,2000-08-19,Chelsea,West Ham,4.0,2.0,1,1,1610,1633,1,1
4,E0,2000-08-19,Coventry,Middlesbrough,1.0,3.0,2,1,4001,4007,2,2
6,E0,2000-08-19,Derby,Southampton,2.0,2.0,0,1,4002,1619,3,3
8,E0,2000-08-19,Leeds,Everton,2.0,0.0,1,1,4003,1623,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
90754,F1,2000-05-13,Nancy,Auxerre,2.0,0.0,1,9900,4123,4115,49493,6830
90756,F1,2000-05-13,Rennes,Metz,2.0,0.0,1,9900,3774,3777,49494,6446
90758,F1,2000-05-13,Sedan,Marseille,2.0,2.0,0,9900,4116,3771,49495,6518
90760,F1,2000-05-13,St Etienne,Monaco,3.0,1.0,1,9900,3782,19830,49496,6519


In [312]:
# creamos dos dataframes (para local y visitantes) donde cada fila es un partido.

df1 = fdata_allseasons_goals[['matchId','Div','season','Date','id_derby']]
df2 = fdata_allseasons_goals[['matchId','Div','season','Date','id_derby']]
df1['Team']     = fdata_allseasons_goals.IdH;     df2['Team']     = fdata_allseasons_goals.IdA
df1['Scored']   = fdata_allseasons_goals.FTHG;    df2['Scored']   = fdata_allseasons_goals.FTAG
df1['Received'] = fdata_allseasons_goals.FTAG;    df2['Received'] = fdata_allseasons_goals.FTHG
df1['Side'] = 0; df2['Side'] = 1

df1['points'] = [getPoints(s,r) for s,r in zip(df1.Scored,df1.Received)]
df2['points'] = [getPoints(s,r) for s,r in zip(df2.Scored,df2.Received)]



In [313]:
# concatenamos los dos datasets y los ordenamos por tiempo (reciente a antiguo) y id de partido

df = pd.concat([df1,df2])
df.sort_values(['Date','matchId'],ascending=True,inplace=True)
df

,matchId,Div,season,Date,id_derby,Team,Scored,Received,Side,points
52468,27627,D1,9394,1993-08-07,3486,2444,3.0,1.0,0,3
52468,27627,D1,9394,1993-08-07,3486,2453,1.0,3.0,1,0
52470,27628,D1,9394,1993-08-07,3925,2447,2.0,1.0,0,3
52470,27628,D1,9394,1993-08-07,3925,4069,1.0,2.0,1,0
52472,27629,D1,9394,1993-08-07,3899,4068,2.0,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...
88922,48331,F1,2021,2021-05-23,7481,4134,0.0,2.0,1,0
88924,48332,F1,2021,2021-05-23,7180,3782,0.0,1.0,0,0
88924,48332,F1,2021,2021-05-23,7180,3804,1.0,0.0,1,3
88926,48333,F1,2021,2021-05-23,6933,3779,1.0,1.0,0,1


In [344]:
npj = range(1,11)
dictDF = {}

for n in npj[:2]:
    ex = (df.sort_values('Date',ascending=True)
        # .groupby(['season','Team'])
        .groupby(['season','Team'])
        .rolling(window=n,min_periods=n,closed='left')
        .agg({'Side':'sum','Scored':'sum','Received':'sum','points':'mean'})
        .swaplevel(0,2,axis=0)
        .reset_index()
        # .merge(df[['matchId','Date','Team','Side']],
                # left_index=True,right_on=[df.matchId,df.Team])
    #     .drop(columns=['key_0','key_1','season','Date','Team'])
    #     .sort_values(['matchId','Side_y'])
        )

    # ex.columns = [f'side_avg_{n}',f'Scored_{n}',f'Received_{n}',f'points_{n}','matchId','Side']

    # df1 = ex[::2]; df2 = ex[1::2]

    # dictDF[n] = (df1.drop(columns=['Side','matchId'])
    #     .join(df2.drop(columns=['matchId','Side']),
    #             lsuffix='_home',rsuffix='_away'))


ex

,level_0,Team,season,Side,Scored,Received,points
0,20032,674,1,NaN,NaN,NaN,NaN
1,20054,674,1,NaN,NaN,NaN,NaN
2,20082,674,1,1.0,5.0,2.0,1.5
3,20102,674,1,1.0,7.0,0.0,3.0
4,20114,674,1,1.0,8.0,1.0,3.0
...,...,...,...,...,...,...,...
90759,90674,19830,9900,1.0,4.0,0.0,3.0
90760,90700,19830,9900,1.0,4.0,4.0,1.5
90761,90718,19830,9900,1.0,4.0,6.0,0.5
90762,90738,19830,9900,1.0,3.0,4.0,0.5


In [343]:
df

,matchId,Div,season,Date,id_derby,Team,Scored,Received,Side,points
52468,27627,D1,9394,1993-08-07,3486,2444,3.0,1.0,0,3
52468,27627,D1,9394,1993-08-07,3486,2453,1.0,3.0,1,0
52470,27628,D1,9394,1993-08-07,3925,2447,2.0,1.0,0,3
52470,27628,D1,9394,1993-08-07,3925,4069,1.0,2.0,1,0
52472,27629,D1,9394,1993-08-07,3899,4068,2.0,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...
88922,48331,F1,2021,2021-05-23,7481,4134,0.0,2.0,1,0
88924,48332,F1,2021,2021-05-23,7180,3782,0.0,1.0,0,0
88924,48332,F1,2021,2021-05-23,7180,3804,1.0,0.0,1,3
88926,48333,F1,2021,2021-05-23,6933,3779,1.0,1.0,0,1


In [324]:
historical_goals = fdata_allseasons_goals.join(list(dictDF.values()))
historical_goals

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,...,Received_9_away,points_9_away,side_avg_10_home,Scored_10_home,Received_10_home,points_10_home,side_avg_10_away,Scored_10_away,Received_10_away,points_10_away
0,E0,2000-08-19,Charlton,Man City,4.0,0.0,1,1,4000,1625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2000-08-19,Chelsea,West Ham,4.0,2.0,1,1,1610,1633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2000-08-19,Coventry,Middlesbrough,1.0,3.0,2,1,4001,4007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,E0,2000-08-19,Derby,Southampton,2.0,2.0,0,1,4002,1619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,E0,2000-08-19,Leeds,Everton,2.0,0.0,1,1,4003,1623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90754,F1,2000-05-13,Nancy,Auxerre,2.0,0.0,1,9900,4123,4115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90756,F1,2000-05-13,Rennes,Metz,2.0,0.0,1,9900,3774,3777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90758,F1,2000-05-13,Sedan,Marseille,2.0,2.0,0,9900,4116,3771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90760,F1,2000-05-13,St Etienne,Monaco,3.0,1.0,1,9900,3782,19830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [325]:
historical_goals.dropna()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,...,Received_9_away,points_9_away,side_avg_10_home,Scored_10_home,Received_10_home,points_10_home,side_avg_10_away,Scored_10_away,Received_10_away,points_10_away
8320,E0,2013-05-11,Aston Villa,Chelsea,1.0,2.0,2,1213,4008,1610,...,2.0,2.444444,5.0,17.0,13.0,1.5,6.0,18.0,2.0,2.5
9536,E0,2015-01-11,Arsenal,Stoke,3.0,0.0,1,1415,1609,1639,...,15.0,1.111111,5.0,22.0,10.0,2.0,5.0,11.0,18.0,1.0
12910,E0,2019-05-12,Liverpool,Wolves,2.0,0.0,1,1819,1612,4018,...,9.0,1.777778,4.0,11.0,19.0,0.9,6.0,14.0,11.0,1.6
15546,E0,1994-10-29,Everton,Arsenal,1.0,1.0,0,9495,1623,1609,...,12.0,1.333333,5.0,19.0,14.0,1.5,5.0,21.0,13.0,1.5
38898,SP1,1997-12-13,Barcelona,Espanol,3.0,1.0,1,9798,676,691,...,6.0,1.666667,4.0,23.0,20.0,1.2,6.0,11.0,8.0,1.5
40102,SP1,1999-06-20,Betis,Ath Bilbao,1.0,4.0,2,9899,684,678,...,15.0,1.333333,5.0,8.0,8.0,1.4,4.0,15.0,16.0,1.5


In [316]:
df.sort_values('Date',ascending=True)

,matchId,Div,season,Date,id_derby,Team,Scored,Received,Side,points
52468,27627,D1,9394,1993-08-07,3486,2444,3.0,1.0,0,3
52484,27635,D1,9394,1993-08-07,3641,2443,5.0,1.0,0,3
52482,27634,D1,9394,1993-08-07,4436,2449,0.0,3.0,1,0
52482,27634,D1,9394,1993-08-07,4436,4080,3.0,0.0,0,3
52480,27633,D1,9394,1993-08-07,4435,2462,4.0,0.0,1,3
...,...,...,...,...,...,...,...,...,...,...
35224,18309,SP1,2021,2021-05-23,2745,698,0.0,0.0,1,1
35224,18309,SP1,2021,2021-05-23,2745,4048,0.0,0.0,0,1
88926,48333,F1,2021,2021-05-23,6933,3779,1.0,1.0,0,1
88914,48327,F1,2021,2021-05-23,6647,3766,2.0,3.0,0,0


In [317]:
# HISTORICAL MATCHES A vs B

npj = [1,3,5,10]

for n in npj:
    ex = (df.sort_values('Date',ascending=True)
        .set_index('matchId')
        .groupby(['id_derby','Team'])
        .rolling(window=n,min_periods=min(2,n),closed='left')
        .agg({'Side':'sum','Scored':'sum','Received':'sum','points':'mean'})
        # .reset_index(level=[2])
        .merge(df[['matchId','Team','Side','id_derby']],on=['matchId','Team'])
    )

    ex.columns = ['matchId', 'Team', 'Side', 'Scored', 'Received', 'points', 's','id_derby']

    ex = ex.pivot(index='matchId',columns='s',values=['Side','Scored','Received','points'])
    ex = ex.T.reorder_levels(order=[1,0]).T
    ex = ex[sorted(ex.columns)]
    colhome = [f'derby_{c2}_{n}_home' for i,(c1,c2) in enumerate(ex.columns) if i<6]
    colaway = [f'derby_{c2}_{n}_away' for i,(c1,c2) in enumerate(ex.columns) if i>=6]
    ex.columns = [*colhome,*colaway]
    print(n,ex.shape)
    if n==1: res = ex
    else: res = res.join(ex,on='matchId')

1 (45382, 8)
3 (45382, 8)
5 (45382, 8)
10 (45382, 8)


In [318]:
res

,derby_Received_1_home,derby_Scored_1_home,derby_Side_1_home,derby_points_1_home,derby_Received_1_home,derby_Scored_1_home,derby_Side_1_away,derby_points_1_away,derby_Received_3_home,derby_Scored_3_home,...,derby_Side_5_away,derby_points_5_away,derby_Received_10_home,derby_Scored_10_home,derby_Side_10_home,derby_points_10_home,derby_Received_10_home,derby_Scored_10_home,derby_Side_10_away,derby_points_10_away
matchId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,2.0,...,0.0,1.6,9.0,9.0,7.0,1.142857,9.0,9.0,0.0,1.571429
2,2.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0,8.0,0.0,...,0.0,3.0,10.0,1.0,4.0,0.000000,1.0,10.0,0.0,3.000000
3,3.0,3.0,1.0,1.0,3.0,3.0,0.0,1.0,3.0,6.0,...,0.0,1.0,6.0,7.0,4.0,1.750000,7.0,6.0,0.0,1.000000
4,4.0,4.0,1.0,1.0,4.0,4.0,0.0,1.0,6.0,4.0,...,0.0,1.8,12.0,5.0,7.0,0.571429,5.0,12.0,0.0,1.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576334,0.0,3.0,0.0,3.0,3.0,0.0,1.0,0.0,1.0,9.0,...,5.0,0.0,4.0,16.0,0.0,3.000000,16.0,4.0,7.0,0.000000
2576335,3.0,1.0,0.0,0.0,1.0,3.0,1.0,3.0,5.0,4.0,...,5.0,1.2,13.0,13.0,0.0,1.600000,13.0,13.0,10.0,1.300000
2576336,3.0,1.0,0.0,0.0,1.0,3.0,1.0,3.0,8.0,1.0,...,4.0,3.0,10.0,1.0,0.0,0.000000,1.0,10.0,4.0,3.000000


In [319]:
# IMPORT CSV and join

In [320]:
# historical_goals = pd.read_csv(path_rawdata+'historical_goals.csv',sep=';',decimal=',',index_col='aux')

In [321]:
historical_goals = historical_goals.join(res,how='left')

In [322]:
historical_goals.dropna()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,...,derby_Side_5_away,derby_points_5_away,derby_Received_10_home,derby_Scored_10_home,derby_Side_10_home,derby_points_10_home,derby_Received_10_home,derby_Scored_10_home,derby_Side_10_away,derby_points_10_away
12910,E0,2019-05-12,Liverpool,Wolves,2.0,0.0,1,1819,1612,4018,...,5.0,0.6,6.0,12.0,0.0,2.111111,12.0,6.0,9.0,0.777778
15546,E0,1994-10-29,Everton,Arsenal,1.0,1.0,0,9495,1623,1609,...,5.0,0.6,5.0,27.0,0.0,2.300000,27.0,5.0,10.0,0.500000


In [272]:
historical_goals.to_csv(path_rawdata+'historical_goals.csv',sep=';',decimal=',',index=True)

##### Historical goals dataset: rolling over time

In [273]:
npj = ['15D','30D','60D']
dictDF = {}
df.Date = pd.to_datetime(df.Date)

for n in npj:
    ex = (df.sort_values('Date',ascending=True)
                .groupby(['season','Team'])
                .rolling(window=n,min_periods=2,on='Date',closed='left')
                .agg({'Side':'sum','Scored':'sum','Received':'sum','points':'mean'})
                .sort_index(level=[1,2])
                .assign(matchId = df.sort_values(['Team','Date']).matchId.to_numpy())
                .set_index('matchId',append=True).reset_index([0,2])
                .merge(df[['matchId','Team','Side']],
                    left_index=True,right_on=[df.Team,df.matchId])
                .drop(columns=['key_0','key_1','season','Date','Team'])
                .sort_values(['matchId','Side_y'])
            )   

    ex.columns = [f'side_avg_{n}',f'Scored_{n}',f'Received_{n}',f'points_{n}','matchId','Side']

    df1 = ex[::2]; df2 = ex[1::2]

    dictDF[n] = (df1.drop(columns=['Side','matchId'])    
        .join(df2.drop(columns=['matchId','Side']),
                lsuffix='_home',rsuffix='_away'))

In [36]:
historical_goals_date = fdata_allseasons_goals.join(list(dictDF.values()))

In [37]:
historical_goals_date.dropna().shape

(29763, 35)

In [274]:
historical_goals_date = pd.read_csv(path_rawdata+'historical_goals_date.csv',sep=';',index_col='aux')
historical_goals_date

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,IdH,IdA,...,Received_30D_away,points_30D_away,side_avg_60D_home,Scored_60D_home,Received_60D_home,points_60D_home,side_avg_60D_away,Scored_60D_away,Received_60D_away,points_60D_away
aux,,,,,,,,,,,,,,,,,,,,,
0,E0,2000-08-19,Charlton,Man City,4.0,0.0,1,1,4000,1625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2000-08-19,Chelsea,West Ham,4.0,2.0,1,1,1610,1633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2000-08-19,Coventry,Middlesbrough,1.0,3.0,2,1,4001,4007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2000-08-19,Derby,Southampton,2.0,2.0,0,1,4002,1619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2000-08-19,Leeds,Everton,2.0,0.0,1,1,4003,1623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49493,F1,2000-05-13,Nancy,Auxerre,2.0,0.0,1,9900,4123,4115,...,4.0,1.333333,3.0,9.0,10.0,1.0,2.0,7.0,7.0,1.0
49494,F1,2000-05-13,Rennes,Metz,2.0,0.0,1,9900,3774,3777,...,2.0,2.333333,3.0,6.0,12.0,0.6,2.0,6.0,4.0,1.6
49495,F1,2000-05-13,Sedan,Marseille,2.0,2.0,0,9900,4116,3771,...,4.0,1.000000,3.0,3.0,6.0,0.8,3.0,9.0,9.0,1.2


In [276]:
historical_goals_date = historical_goals_date.join(res,how='left')

In [277]:
historical_goals_date.to_csv(path_rawdata+'historical_goals_date.csv',sep=';',decimal=',',index=True)

##### Historical goals dataset: long term memory

In [39]:
npj     = ['180D','365D','730D','1825D']
min_n   = [5,10,15,15] # CHANGE TO EXPAND DATASET  

for n,m in zip(npj,min_n):
    ex = (df.sort_values('Date',ascending=True)
                .groupby('Team')
                .rolling(window=n,min_periods=m,on='Date',closed='left')
                .agg({'Side':'sum','Scored':'sum','Received':'sum','points':'mean'})
                .sort_index(level=[0,1])
                .assign(matchId = df.sort_values(['Team','Date']).matchId.to_numpy())
                .set_index('matchId',append=True).reset_index(1)
                .merge(df[['matchId','Team','Side']],
                    left_index=True,right_on=[df.Team,df.matchId])
                .drop(columns=['key_0','key_1','Date','Team'])
                .sort_values(['matchId','Side_y'])
            )   

    ex.columns = [f'side_avg_{n}',f'Scored_{n}',f'Received_{n}',f'points_{n}','matchId','Side']

    df1 = ex[::2]; df2 = ex[1::2]

    dictDF[n] = (df1.drop(columns=['Side','matchId'])    
        .join(df2.drop(columns=['matchId','Side']),
                lsuffix='_home',rsuffix='_away'))

In [40]:
historical_goals_longterm = fdata_allseasons_goals.join(list(dictDF.values()))

In [41]:
historical_goals_longterm.dropna().shape

(26598, 67)

In [285]:
historical_goals_longterm = pd.read_csv(path_rawdata+'historical_goals_longterm.csv',sep=';',index_col='aux')

In [286]:
historical_goals_longterm = historical_goals_longterm.join(res,how='left')
historical_goals_longterm

ValueError: columns overlap but no suffix specified: Index(['derby_Received_1_home', 'derby_Scored_1_home', 'derby_Side_1_home',
       'derby_Team_1_home', 'derby_points_1_home', 'derby_Scored_1_away',
       'derby_Side_1_away', 'derby_Team_1_away', 'derby_points_1_away',
       'derby_Received_3_home', 'derby_Scored_3_home', 'derby_Side_3_home',
       'derby_Team_3_home', 'derby_points_3_home', 'derby_Scored_3_away',
       'derby_Side_3_away', 'derby_Team_3_away', 'derby_points_3_away',
       'derby_Received_5_home', 'derby_Scored_5_home', 'derby_Side_5_home',
       'derby_Team_5_home', 'derby_points_5_home', 'derby_Scored_5_away',
       'derby_Side_5_away', 'derby_Team_5_away', 'derby_points_5_away',
       'derby_Received_10_home', 'derby_Scored_10_home', 'derby_Side_10_home',
       'derby_Team_10_home', 'derby_points_10_home', 'derby_Scored_10_away',
       'derby_Side_10_away', 'derby_Team_10_away', 'derby_points_10_away'],
      dtype='object')

In [ ]:
historical_goals_longterm.to_csv(path_rawdata+'historical_goals_longterm.csv',decimal=',',sep=';',index=True)

## Historical dataset: more attributes (less rows)

In [43]:
null_HST = fdata_allseasons_clean[fdata_allseasons_clean.HST.isna()]

for div in np.unique(null_HST.Div):
    print(div, len(null_HST[null_HST.Div==div]))

D1 2754
E0 2824
F1 1912
I1 3075
SP1 4344


In [44]:
fdata_allseasons_dropna = fdata_allseasons_clean.dropna(axis=0)

print(len(fdata_allseasons_dropna) / len(fdata_allseasons))
print()
for div in np.unique(fdata_allseasons_dropna.Div):
    print(div, len(fdata_allseasons_dropna[fdata_allseasons_dropna.Div==div]))
print()
for season in np.unique(fdata_allseasons_dropna.season):
    print(season, len(fdata_allseasons_dropna[fdata_allseasons_dropna.season==season]))

0.5972780636860727

D1 5184
E0 7190
F1 5216
I1 6041
SP1 6080

1 686
102 380
203 686
405 380
506 746
607 1446
708 1826
809 1826
910 1826
1011 1826
1112 1824
1213 1825
1314 1826
1415 1825
1516 1826
1617 1824
1718 1826
1819 1826
1920 1725
2021 1756


In [45]:
formatDatesDF(fdata_allseasons_dropna)

In [46]:
# fdata_allseasons_dropna['IdH'] = [getId(name) for name in fdata_allseasons_dropna.HomeTeam]
# fdata_allseasons_dropna['IdA'] = [getId(name) for name in fdata_allseasons_dropna.AwayTeam]

In [47]:
# sustituir Gladbach y Villareal
# 4078 por 2454
# 4057 por 682
fdata_allseasons_dropna.IdH[fdata_allseasons_dropna.IdH==4078] = 2454
fdata_allseasons_dropna.IdA[fdata_allseasons_dropna.IdA==4078] = 2454
fdata_allseasons_dropna.IdH[fdata_allseasons_dropna.IdH==4057] = 682
fdata_allseasons_dropna.IdA[fdata_allseasons_dropna.IdA==4057] = 682

In [48]:
fdata_allseasons_dropna.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY',
       'AY', 'HR', 'AR', 'season', 'IdH', 'IdA', 'matchId'],
      dtype='object')

In [49]:
# creamos dos dataframes (para local y visitantes) donde cada fila es un partido.

df1 = fdata_allseasons_dropna[['matchId','Div','season','Date']]
df2 = fdata_allseasons_dropna[['matchId','Div','season','Date']]
df1['Team']     = fdata_allseasons_dropna.IdH;     df2['Team']     = fdata_allseasons_dropna.IdA
df1['Scored']   = fdata_allseasons_dropna.FTHG;    df2['Scored']   = fdata_allseasons_dropna.FTAG
df1['Received'] = fdata_allseasons_dropna.FTAG;    df2['Received'] = fdata_allseasons_dropna.FTHG
df1['Shots']    = fdata_allseasons_dropna.HS;      df2['Shots']    = fdata_allseasons_dropna.AS
df1['Target']   = fdata_allseasons_dropna.HST;     df2['Target']   = fdata_allseasons_dropna.AST
df1['Corner']   = fdata_allseasons_dropna.HC;      df2['Corner']   = fdata_allseasons_dropna.AC
df1['Faults']   = fdata_allseasons_dropna.HF;      df2['Faults']   = fdata_allseasons_dropna.AF
df1['YellowCards']   = fdata_allseasons_dropna.HY;     df2['YellowCards']   = fdata_allseasons_dropna.AY
df1['RedCards'] = fdata_allseasons_dropna.HY;      df2['RedCards'] = fdata_allseasons_dropna.AR
df1['Side'] = 0; df2['Side'] = 1

df1['points'] = [getPoints(s,r) for s,r in zip(df1.Scored,df1.Received)]
df2['points'] = [getPoints(s,r) for s,r in zip(df2.Scored,df2.Received)]

# concatenamos los dos datasets y los ordenamos por tiempo (reciente a antiguo) y id de partido

df = pd.concat([df1,df2])
df.sort_values(['Date','matchId'],ascending=True,inplace=True)
df

,matchId,Div,season,Date,Team,Scored,Received,Shots,Target,Corner,Faults,YellowCards,RedCards,Side,points
aux,,,,,,,,,,,,,,,
21831,21831,D1,1,2000-08-11,2447,1.0,0.0,17.0,7.0,7.0,25.0,1.0,1.0,0,3
21831,21831,D1,1,2000-08-11,4064,0.0,1.0,5.0,2.0,3.0,19.0,5.0,0.0,1,0
21832,21832,D1,1,2000-08-12,2444,4.0,1.0,14.0,6.0,4.0,13.0,1.0,1.0,0,3
21832,21832,D1,1,2000-08-12,2457,1.0,4.0,11.0,5.0,9.0,12.0,0.0,0.0,1,0
21833,21833,D1,1,2000-08-12,2453,4.0,0.0,15.0,7.0,4.0,22.0,1.0,1.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48331,48331,F1,2021,2021-05-23,4134,0.0,2.0,9.0,2.0,3.0,10.0,0.0,0.0,1,0
48332,48332,F1,2021,2021-05-23,3782,0.0,1.0,19.0,5.0,9.0,13.0,1.0,1.0,0,0
48332,48332,F1,2021,2021-05-23,3804,1.0,0.0,10.0,2.0,3.0,10.0,0.0,0.0,1,3


In [50]:
npj = range(1,11)
dictDF = {}

aggregation = {'Side':'sum','Scored':'sum','Received':'sum','points':'mean','Shots':'mean',
                'Target':'mean','Corner':'mean','Faults':'mean','YellowCards':'mean','RedCards':'mean'}

for n in npj:
    ex = (df.sort_values('Date',ascending=True)
        .groupby(['season','Team'])
        .rolling(window=n,min_periods=n,closed='left')
        .agg(aggregation)
        .swaplevel(0,2,axis=0)
        .reset_index(level=[2])
        .merge(df[['matchId','Date','Team','Side']],
                left_index=True,right_on=[df.matchId,df.Team])
        .drop(columns=['key_0','key_1','season','Date','Team'])
        .sort_values(['matchId','Side_y']))

    ex.columns = [f'side_avg_{n}',f'Scored_{n}',f'Received_{n}',f'points_{n}',f'Shots_{n}',f'Target_{n}'
                    ,f'Corner_{n}',f'Faults_{n}',f'YellowCards_{n}',f'RedCards_{n}','matchId','Side']

    df1 = ex[::2]; df2 = ex[1::2]

    dictDF[n] = (df1.drop(columns=['Side','matchId'])    
        .join(df2.drop(columns=['matchId','Side']),
                lsuffix='_home',rsuffix='_away'))

In [51]:
historical = fdata_allseasons_dropna.join(list(dictDF.values()))
historical

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,side_avg_10_away,Scored_10_away,Received_10_away,points_10_away,Shots_10_away,Target_10_away,Corner_10_away,Faults_10_away,YellowCards_10_away,RedCards_10_away
aux,,,,,,,,,,,,,,,,,,,,,
0,E0,2000-08-19,Charlton,Man City,4.0,0.0,1,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,2000-08-19,Chelsea,West Ham,4.0,2.0,1,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,2000-08-19,Coventry,Middlesbrough,1.0,3.0,2,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,2000-08-19,Derby,Southampton,2.0,2.0,0,1.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,2000-08-19,Leeds,Everton,2.0,0.0,1,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48329,F1,2021-05-23,Nantes,Montpellier,1.0,2.0,2,1.0,1.0,D,...,5.0,15.0,15.0,1.2,12.0,4.2,4.4,14.7,1.4,0.7
48330,F1,2021-05-23,Reims,Bordeaux,1.0,2.0,2,1.0,1.0,D,...,5.0,12.0,22.0,0.9,10.8,3.8,4.5,12.7,2.5,1.4
48331,F1,2021-05-23,Rennes,Nimes,2.0,0.0,1,1.0,0.0,H,...,5.0,14.0,17.0,1.0,12.0,4.3,4.5,11.6,1.7,0.9


In [52]:
historical.to_csv(path_rawdata+'historical.csv',sep=';',index=True)

##### Historical dataset: rolling over time

In [53]:
npj = ['15D','30D','60D']
dictDF = {}

df.Date = pd.to_datetime(df.Date)

aggregation = {'Side':'sum','Scored':'sum','Received':'sum','points':'mean','Shots':'mean',
                'Target':'mean','Corner':'mean','Faults':'mean','YellowCards':'mean','RedCards':'mean'}

for n in npj:
    ex = (df.sort_values('Date',ascending=True)
        .groupby(['season','Team'])
        .rolling(window=n,min_periods=2,on='Date',closed='left')
        .agg(aggregation)
        .sort_index(level=[1,2])
        .assign(matchId = df.sort_values(['Team','Date']).matchId.to_numpy())
        .set_index('matchId',append=True).reset_index([0,2])
        .merge(df[['matchId','Team','Side']],
                left_index=True,right_on=[df.Team,df.matchId])
        .drop(columns=['key_0','key_1','season','Date','Team'])
        .sort_values(['matchId','Side_y'])
        )

    ex.columns = [f'side_avg_{n}',f'Scored_{n}',f'Received_{n}',f'points_{n}',f'Shots_{n}',f'Target_{n}'
                    ,f'Corner_{n}',f'Faults_{n}',f'YellowCards_{n}',f'RedCards_{n}','matchId','Side']

    df1 = ex[::2]; df2 = ex[1::2]

    dictDF[n] = (df1.drop(columns=['Side','matchId'])    
        .join(df2.drop(columns=['matchId','Side']),
                lsuffix='_home',rsuffix='_away'))

In [54]:
historical_date = fdata_allseasons_dropna.join(list(dictDF.values()))
historical_date.shape

(29711, 86)

In [55]:
historical_date.to_csv(path_rawdata+'historical_date.csv',sep=';',index=True)

##### Historical dataset: long term memory

In [56]:
npj     = ['180D','365D','730D','1825D']
min_n   = [5,10,15,15] # CHANGE TO EXPAND DATASET  

for n,m in zip(npj,min_n):
    ex = (df.sort_values('Date',ascending=True)
        .groupby('Team')
        .rolling(window=n,min_periods=m,on='Date',closed='left')
        .agg(aggregation)
        .sort_index(level=[0,1])
        .assign(matchId = df.sort_values(['Team','Date']).matchId.to_numpy())
        .set_index('matchId',append=True).reset_index(1)
        .merge(df[['matchId','Team','Side']],
                left_index=True,right_on=[df.Team,df.matchId])
        .drop(columns=['key_0','key_1','Date','Team'])
        .sort_values(['matchId','Side_y'])
        )

    ex.columns = [f'side_avg_{n}',f'Scored_{n}',f'Received_{n}',f'points_{n}',f'Shots_{n}',f'Target_{n}'
                    ,f'Corner_{n}',f'Faults_{n}',f'YellowCards_{n}',f'RedCards_{n}','matchId','Side']

    df1 = ex[::2]; df2 = ex[1::2]

    dictDF[n] = (df1.drop(columns=['Side','matchId'])    
        .join(df2.drop(columns=['matchId','Side']),
                lsuffix='_home',rsuffix='_away'))

In [57]:
historical_longterm = fdata_allseasons_dropna.join(list(dictDF.values()))

In [58]:
historical_longterm.shape

(29711, 166)

In [59]:
historical_longterm.to_csv(path_rawdata+'historical_longterm.csv',sep=';',index=True)

## READ DATA

In [60]:
hist_goals_long = pd.read_csv(path_rawdata+'historical_longterm.csv',sep=';',index_col='matchId') 

In [61]:
hist_goals_long.shape

(29711, 166)

In [62]:
hist_goals_long.columns[:]

Index(['aux', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG',
       ...
       'side_avg_1825D_away', 'Scored_1825D_away', 'Received_1825D_away',
       'points_1825D_away', 'Shots_1825D_away', 'Target_1825D_away',
       'Corner_1825D_away', 'Faults_1825D_away', 'YellowCards_1825D_away',
       'RedCards_1825D_away'],
      dtype='object', length=166)

In [63]:
hist_goals_long.dropna()

,aux,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,side_avg_1825D_away,Scored_1825D_away,Received_1825D_away,points_1825D_away,Shots_1825D_away,Target_1825D_away,Corner_1825D_away,Faults_1825D_away,YellowCards_1825D_away,RedCards_1825D_away
matchId,,,,,,,,,,,,,,,,,,,,,
149,149,E0,2000-12-02,Arsenal,Southampton,1.0,0.0,1,0.0,0.0,...,7.0,21.0,27.0,1.133333,11.266667,5.200000,6.666667,12.800000,1.466667,0.866667
151,151,E0,2000-12-02,Bradford,Coventry,2.0,1.0,1,0.0,0.0,...,7.0,14.0,30.0,0.800000,7.866667,3.733333,4.400000,12.800000,1.600000,0.800000
152,152,E0,2000-12-02,Ipswich,Derby,0.0,1.0,2,0.0,1.0,...,7.0,18.0,31.0,0.666667,8.666667,3.866667,4.533333,13.533333,2.200000,1.066667
154,154,E0,2000-12-02,Liverpool,Charlton,3.0,0.0,1,1.0,0.0,...,7.0,21.0,21.0,1.400000,9.000000,5.466667,5.866667,12.133333,1.266667,0.666667
155,155,E0,2000-12-02,Man United,Tottenham,2.0,0.0,1,1.0,0.0,...,7.0,22.0,21.0,1.533333,11.200000,5.066667,5.133333,13.933333,1.266667,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48329,48329,F1,2021-05-23,Nantes,Montpellier,1.0,2.0,2,1.0,1.0,...,89.0,230.0,236.0,1.340782,12.927374,4.335196,4.787709,12.865922,1.664804,0.960894
48330,48330,F1,2021-05-23,Reims,Bordeaux,1.0,2.0,2,1.0,1.0,...,90.0,220.0,222.0,1.307263,11.525140,3.871508,4.564246,13.117318,1.921788,0.938547
48331,48331,F1,2021-05-23,Rennes,Nimes,2.0,0.0,1,1.0,0.0,...,51.0,126.0,171.0,1.116505,12.165049,4.145631,4.883495,12.213592,1.728155,0.922330


In [34]:
import importlib as il
import mytrain_lib_cluster as ml
il.reload(ml)
traindata = ml.FootballMatchesDataset('train','historical_date',drop=[],factor=-1)
testdata = ml.FootballMatchesDataset('test','historical',drop=[],factor=0)

In [35]:
traindata.data.shape, testdata.data.shape

(torch.Size([15416, 60]), torch.Size([4348, 200]))

In [33]:
traindata.labels.max(axis=0)

torch.return_types.max(
values=tensor([1., 1., 1.]),
indices=tensor([1, 0, 9]))